# Create IRs for D-CASE workshop

- Get 5 rooms from Gibson dataset
- Place an Eigenmike32 in the center of the room
- Uniformly place 20 sources in the *same room*
- Grab the IRs and sound/microphone locations and save as numpy arrays

## Import dependencies, set constants

In [36]:
import glob
import os
import random

import numpy as np
import h5py
from loguru import logger

from audiblelight import utils
from audiblelight.space import Space

In [2]:
# For reproducible random source placement
utils.seed_everything(utils.SEED)

In [37]:
N_ROOMS = 5
N_SOURCES = 20
MIC_TYPE = "eigenmike32"
OUT_DIR = os.path.join(utils.get_project_root(), "resources/dcase_irs")

In [38]:
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

## Get 5 random rooms from the Gibson dataset

This probably won't work unless you have the Gibson dataset files saved inside `resources/meshes`

In [4]:
# Get all the object files we're considering here
dataset_dir = os.path.join(utils.get_project_root(), "resources/meshes")
mesh_paths = glob.glob('**/*.glb', root_dir=dataset_dir)

In [5]:
gibsons = [os.path.join(dataset_dir, r) for r in random.sample(mesh_paths, N_ROOMS)]

## Load up the rooms in AudibleLight

In [6]:
spaces = [Space(sp) for sp in gibsons]

Material for category 'default' was not found. Using default material instead.
Material for category 'default' was not found. Using default material instead.
Material for category 'default' was not found. Using default material instead.
Material for category 'default' was not found. Using default material instead.
Material for category 'default' was not found. Using default material instead.


## Add Eigenmike32s to the spaces

We just place the microphone in a random position for now

In [7]:
for space in spaces:
    space.add_microphones(MIC_TYPE, keep_existing=False)

## Place the sound sources

In [15]:
# This places 20 sources and ensures that all have a direct line to our mic
for space_idx, space in enumerate(spaces, 1):
    logger.info(f"Placing sources in space {space_idx}")
    space.add_sources(N_SOURCES, keep_existing=False, ensure_direct_line=True, mic_idx=0)
    # Sanity checking
    assert len(space.source_positions == N_SOURCES)
    for source in space.source_positions:
        assert space.direct_line_exists_between_points(source, space.microphones[0].coordinates_center)

2025-07-02 10:40:25.403 | INFO     | __main__:<module>:3 - Placing sources in space 1
2025-07-02 10:40:52.481 | WARNING  | audiblelight.space:calculate_weighted_average_ray_length:184 - Some rays cast from point [4.7478294  1.1575996  2.04331207] have infinite distances: is the mesh watertight?
2025-07-02 10:41:12.684 | INFO     | __main__:<module>:3 - Placing sources in space 2
2025-07-02 10:41:19.288 | WARNING  | audiblelight.space:calculate_weighted_average_ray_length:184 - Some rays cast from point [ 4.70550067 -2.2388111   1.811053  ] have infinite distances: is the mesh watertight?
2025-07-02 10:41:23.823 | WARNING  | audiblelight.space:calculate_weighted_average_ray_length:184 - Some rays cast from point [ 3.94845453 -3.89669362  0.62815019] have infinite distances: is the mesh watertight?
2025-07-02 10:41:26.668 | WARNING  | audiblelight.space:calculate_weighted_average_ray_length:184 - Some rays cast from point [ 2.60909341 -4.88716083  0.70831383] have infinite distances: is 

In [ ]:
# Check the microphone and source positioning
scn = spaces[1].create_scene(mic_radius=0.02)
# The individual microphone capsules are shown as red spheres in the mesh
scn.show()

## Run the simulations

In [20]:
for space_idx, space in enumerate(spaces):
    logger.info(f"Simulating space {space_idx}")
    space.simulate()

2025-07-02 10:49:46.035 | INFO     | __main__:<module>:2 - Simulating space 0
2025-07-02 10:55:50.490 | INFO     | audiblelight.space:simulate:678 - Finished simulation! Overall indirect ray efficiency: 0.990
2025-07-02 10:55:52.175 | INFO     | __main__:<module>:2 - Simulating space 1
2025-07-02 10:58:37.829 | INFO     | audiblelight.space:simulate:678 - Finished simulation! Overall indirect ray efficiency: 0.933
2025-07-02 10:58:39.261 | INFO     | __main__:<module>:2 - Simulating space 2
2025-07-02 11:00:18.921 | INFO     | audiblelight.space:simulate:678 - Finished simulation! Overall indirect ray efficiency: 0.878
2025-07-02 11:00:20.365 | INFO     | __main__:<module>:2 - Simulating space 3
2025-07-02 11:04:41.619 | INFO     | audiblelight.space:simulate:678 - Finished simulation! Overall indirect ray efficiency: 0.985
2025-07-02 11:04:43.559 | INFO     | __main__:<module>:2 - Simulating space 4
2025-07-02 11:09:42.415 | INFO     | audiblelight.space:simulate:678 - Finished simula

## Save the IRs

In [62]:
for space_idx, space in enumerate(spaces):
    # Grab IRs
    irs = space.microphones[0].irs
    # Sanity checking
    n_caps, n_sources, n_samples = irs.shape
    assert n_caps == 32
    assert n_sources == N_SOURCES
    # Create the dictionary: keys are strings, values are arrays
    fname = f"room{str(space_idx).zfill(3)}.h5"
    data = dict(
        # **space.mesh.metadata,
        mic_center_position=space.microphones[0].coordinates_center,
        mic_capsules_position=space.microphones[0].coordinates_absolute,
        source_position=space.source_positions,
        mesh_bounds=space.mesh.bounds,
        irs=irs
    )
    # Dump the H5 file
    with h5py.File(os.path.join(OUT_DIR, fname), 'w') as f:
        for key, arr in data.items():
            f.create_dataset(key, data=arr)

## Sanity check loading H5 objects

In [63]:
# Loading the dictionary up
loaded_dict = {}
with h5py.File(os.path.join(OUT_DIR, "room004.h5"), 'r') as f:
    for key in f.keys():
        loaded_dict[key] = f[key][:]

In [64]:
# Check that all arrays are equal to the initial dictionary
for k in loaded_dict.keys():
    assert np.array_equal(loaded_dict[k], data[k])